<a href="https://colab.research.google.com/github/agntgalahad/pothole-detection/blob/main/Pothole-Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import tensorflow as tf
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, RandomFlip
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import files
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d atulyakumar98/pothole-detection-dataset

pothole-detection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [51]:
import zipfile
!mkdir dataset
with zipfile.ZipFile('/content/pothole-detection-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset/')

mkdir: cannot create directory ‘dataset’: File exists


In [52]:
from pathlib import Path
import imghdr
import os

img_link=list(Path("/content/dataset/normal").glob(r'**/*.jpg'))

count_num=0
for lnk in img_link:
    binary_img=open(lnk,'rb')
    find_img=tf.compat.as_bytes('JFIF') in binary_img.peek(10)#The JFIF is a JPEG File Interchange Format (JFIF). It is a standard which we gauge if an image is corrupt or substandard
    if not find_img:
        count_num+=1
        os.remove(str(lnk))
print('Total %d pcs image delete from Dataset' % count_num)

Total 59 pcs image delete from Dataset


In [53]:
img_link=list(Path("/content/dataset/potholes").glob(r'**/*.jpg'))

count_num=0
for lnk in img_link:
    binary_img=open(lnk,'rb')
    find_img=tf.compat.as_bytes('JFIF') in binary_img.peek(10)#The JFIF is a JPEG File Interchange Format (JFIF). It is a standard which we gauge if an image is corrupt or substandard
    if not find_img:
        count_num+=1
        os.remove(str(lnk))
print('Total %d pcs image delete from Dataset' % count_num)

Total 31 pcs image delete from Dataset


In [54]:
img_height, img_width = 227, 227
batch_size = 32
data_dir = '/content/dataset'
class_names = ['normal', 'potholes']

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = 'training',
    label_mode = 'binary',
    batch_size = batch_size,
    seed = 123,
    image_size=(img_height, img_width),
)
print(train_ds)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = 'validation',
    label_mode = 'binary',
    batch_size = batch_size,
    seed = 123,
    image_size=(img_height, img_width),
)
print(valid_ds)

Found 591 files belonging to 2 classes.
Using 473 files for training.
<BatchDataset element_spec=(TensorSpec(shape=(None, 227, 227, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>
Found 591 files belonging to 2 classes.
Using 118 files for validation.
<BatchDataset element_spec=(TensorSpec(shape=(None, 227, 227, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>


In [55]:
model = Sequential()

model.add(RandomFlip('horizontal_and_vertical'))
model.add(Conv2D(filters=32, kernel_size=(5,5), strides=(11,11), input_shape=(img_height, img_width, 3)))
model.add(Activation('relu'))

model.add(Conv2D(filters=32, kernel_size=(3,3), strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=128))
model.add(Activation('relu'))
model.add(Dense(units=1, activation='sigmoid'))



In [56]:
model.build(input_shape=(None,227,227,3))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_10 (RandomFlip)  (None, 227, 227, 3)      0         
                                                                 
 conv2d_20 (Conv2D)          (None, 21, 21, 32)        2432      
                                                                 
 activation_23 (Activation)  (None, 21, 21, 32)        0         
                                                                 
 conv2d_21 (Conv2D)          (None, 10, 10, 32)        9248      
                                                                 
 flatten_10 (Flatten)        (None, 3200)              0         
                                                                 
 dense_20 (Dense)            (None, 128)               409728    
                                                                 
 activation_24 (Activation)  (None, 128)             

In [57]:
history = model.fit(train_ds, epochs =10 , batch_size=batch_size, verbose=2)

Epoch 1/10
15/15 - 8s - loss: 11.0710 - accuracy: 0.5539 - 8s/epoch - 515ms/step
Epoch 2/10
15/15 - 9s - loss: 4.2821 - accuracy: 0.6638 - 9s/epoch - 601ms/step
Epoch 3/10
15/15 - 6s - loss: 2.8030 - accuracy: 0.7463 - 6s/epoch - 370ms/step
Epoch 4/10
15/15 - 6s - loss: 2.3197 - accuracy: 0.7780 - 6s/epoch - 370ms/step
Epoch 5/10
15/15 - 6s - loss: 2.1138 - accuracy: 0.7674 - 6s/epoch - 371ms/step
Epoch 6/10
15/15 - 6s - loss: 1.9017 - accuracy: 0.7801 - 6s/epoch - 367ms/step
Epoch 7/10
15/15 - 5s - loss: 1.4985 - accuracy: 0.8203 - 5s/epoch - 366ms/step
Epoch 8/10
15/15 - 5s - loss: 1.3772 - accuracy: 0.8097 - 5s/epoch - 366ms/step
Epoch 9/10
15/15 - 6s - loss: 1.0705 - accuracy: 0.8436 - 6s/epoch - 376ms/step
Epoch 10/10
15/15 - 6s - loss: 1.1093 - accuracy: 0.8372 - 6s/epoch - 370ms/step


In [58]:
model.evaluate(valid_ds, batch_size = batch_size, verbose=2)

4/4 - 1s - loss: 1.3548 - accuracy: 0.7712 - 781ms/epoch - 195ms/step


[1.354836106300354, 0.7711864113807678]